In [1]:
# %matplotlib notebook
import pandas as pd
import numpy as np
import multiprocessing
import argparse
from datetime import timedelta
import pyvips
from PIL import Image
import time
import random
import gc
import logging
import os
from pathlib import Path
import sys
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

sys.path.append('..')
from src.wsi import slides, filters, tiles, util
from src import dataset, dataloading

BASE_PATH = Path(os.path.dirname(os.getcwd()))
DATA_PATH = BASE_PATH / 'data' / 'originals'
PROCESSED_DATA = BASE_PATH / 'data' / 'processed'

In [10]:
low = pd.read_csv(BASE_PATH / 'data' / 'lowres.csv')
hi = pd.read_csv(BASE_PATH / 'data' / 'hires.csv')
# row = train.iloc[2]
# image = pyvips.Image.new_from_file(f'../data/originals/{row["filename"]}', access='sequential')
# image.filename, image.width, image.height

In [11]:
low = low.set_index('filename')
test = low.sample(frac=1)

In [17]:
test = test.reindex(low.index)

In [18]:
test

,age,sex,body_site,melanoma_history,breslow,ulceration,resolution,tif_cksum,tif_size,us_tif_url,eu_tif_url,as_tif_url
filename,,,,,,,,,,,,
1u4lhlqb.tif,[32:34[,2,thigh,YES,<0.8,NO,0.264384,3028450373,747151312,s3://drivendata-competition-visiomel-public-us/images/1u4lhlqb.tif,s3://drivendata-competition-visiomel-public-eu/images/1u4lhlqb.tif,s3://drivendata-competition-visiomel-public-as/images/1u4lhlqb.tif
rqumqnfp.tif,[46:48[,1,trunc,NO,[1 : 2[,NO,0.264384,1294832049,591027450,s3://drivendata-competition-visiomel-public-us/images/rqumqnfp.tif,s3://drivendata-competition-visiomel-public-eu/images/rqumqnfp.tif,s3://drivendata-competition-visiomel-public-as/images/rqumqnfp.tif
bu5xt1xm.tif,[64:66[,2,face,NO,<0.8,NO,0.264384,774102360,465947458,s3://drivendata-competition-visiomel-public-us/images/bu5xt1xm.tif,s3://drivendata-competition-visiomel-public-eu/images/bu5xt1xm.tif,s3://drivendata-competition-visiomel-public-as/images/bu5xt1xm.tif
dibvu7wk.tif,[62:64[,2,forearm,NaN,[2 : 4[,YES,0.226490,515827065,568174704,s3://drivendata-competition-visiomel-public-us/images/dibvu7wk.tif,s3://drivendata-competition-visiomel-public-eu/images/dibvu7wk.tif,s3://drivendata-competition-visiomel-public-as/images/dibvu7wk.tif
qsza4coh.tif,[90:92[,2,face,NO,[1 : 2[,NO,0.226490,1541795099,1042691978,s3://drivendata-competition-visiomel-public-us/images/qsza4coh.tif,s3://drivendata-competition-visiomel-public-eu/images/qsza4coh.tif,s3://drivendata-competition-visiomel-public-as/images/qsza4coh.tif
...,...,...,...,...,...,...,...,...,...,...,...,...
n7jd638y.tif,[68:70[,1,head/neck,NaN,[1 : 2[,NO,0.250000,1949851255,136729436,s3://drivendata-competition-visiomel-public-us/images/n7jd638y.tif,s3://drivendata-competition-visiomel-public-eu/images/n7jd638y.tif,s3://drivendata-competition-visiomel-public-as/images/n7jd638y.tif
8wg601oe.tif,[66:68[,1,head/neck,NaN,<0.8,NaN,0.250000,1344207528,1603484314,s3://drivendata-competition-visiomel-public-us/images/8wg601oe.tif,s3://drivendata-competition-visiomel-public-eu/images/8wg601oe.tif,s3://drivendata-competition-visiomel-public-as/images/8wg601oe.tif
1ejfd01b.tif,[80:82[,1,trunk,NaN,>=4,YES,0.250000,4292211076,1423564688,s3://drivendata-competition-visiomel-public-us/images/1ejfd01b.tif,s3://drivendata-competition-visiomel-public-eu/images/1ejfd01b.tif,s3://drivendata-competition-visiomel-public-as/images/1ejfd01b.tif


In [3]:
names = !ls ../workspace/tile_data/48
names = [x.split('-')[0] for x in names]

In [13]:
from glob import glob
set1 = set(hi['filename'].str.split('.').str[0])
set2 = set(names)
dels = set2.intersection(set1)
len(dels)
for file in dels:
    f = f'../workspace/tile_data/48/{file}-tile_data.csv'
    os.remove(f)
    f = f'../workspace/tile_data/64/{file}-tile_data.csv'
    os.remove(f)
    f = f'../workspace/tiles/data/processed/*/*/{file}*'
    files = glob(f)
    for fil in files:
        os.remove(fil)

In [3]:
image.get('bands'), image.get('coding'), image.get('format'), image.get('interpretation')

(3, 'none', 'uchar', 'srgb')

In [8]:
image.tiffsave('test.tif', compression="deflate",
                 tile=True, tile_width=256, tile_height=256,
                 pyramid=True)

In [56]:
def rimages(index, train, page):
    filename, res = train.loc[index, ['filename', 'resolution']]
    resizing_factor = res / 0.25
    image = pyvips.Image.new_from_file(f'../data/originals/{filename}', page=page)
    width, height = image.width, image.height
    image = image.affine((resizing_factor, 0, 0, resizing_factor))
    return width, height, image, image.width, image.height

In [3]:
images = pyvips.Image.new_from_file('../data/originals/qmsqv731.tif', page=8)
images.height, images.width
img = images.write_to_file('test.jpg')
# img = np.ndarray(buffer=images.write_to_memory(), dtype=np.uint8, shape=[images.height, images.width, images.bands])
plt.imshow(plt.imread('test.jpg'))


In [10]:
plt.imshow(images.numpy() / 255.)
plt.show()

In [3]:
img = Image.open('test.jpg')
plt.imshow(img)

In [8]:
train.iloc[:, :4].to_csv('../data/train.csv', index=False)

In [3]:
dftrain = dataset.get_dftrain('../workspace/tiles/data/processed/48/3/', 5, 42, 0)
dftrain.dtypes

training dataset: 1252 samples
training dataset: 117941 tiles


tissue_id            object
filename             object
relapse               int64
split                 int64
n_tiles               int64
path                 object
tile_id               int16
is_valid              int64
age                 float64
sex                   int64
melanoma_history      int64
dtype: object

In [79]:
df = dftrain.groupby('tissue_id').agg({'tile_id': list, 'path': list, 'filename': 'first', 'melanoma_history': 'mean', 'sex': 'mean', 'age': 'mean', 'is_valid': 'mean', 'n_tiles': 'mean', 'split': 'mean', 'relapse': 'mean'}).reset_index()
df.dtypes

tissue_id            object
tile_id              object
path                 object
filename             object
melanoma_history    float64
sex                 float64
age                 float64
is_valid            float64
n_tiles             float64
split               float64
relapse             float64
dtype: object

In [88]:
dstrain = dataloading.Dataset('../workspace/tiles/data/processed/48/2/', 5, 42, 0, 80, 16, 256, isval=False)

training dataset: 1252 samples
training dataset: 97504 tiles


In [89]:
dltrain = DataLoader(dstrain, 1, shuffle=True)

In [91]:
image, meta, label = next(iter(dltrain))
meta.dtype

torch.float32

In [4]:
%matplotlib inline

In [26]:
df = pd.read_csv('../data/train_metadata.csv')
df

,filename,age,sex,body_site,melanoma_history,breslow,ulceration,resolution,tif_cksum,tif_size,us_tif_url,eu_tif_url,as_tif_url
0,1u4lhlqb.tif,[32:34[,2,thigh,YES,<0.8,NO,0.264384,3028450373,747151312,s3://drivendata-competition-visiomel-public-us/images/1u4lhlqb.tif,s3://drivendata-competition-visiomel-public-eu/images/1u4lhlqb.tif,s3://drivendata-competition-visiomel-public-as/images/1u4lhlqb.tif
1,rqumqnfp.tif,[46:48[,1,trunc,NO,[1 : 2[,NO,0.264384,1294832049,591027450,s3://drivendata-competition-visiomel-public-us/images/rqumqnfp.tif,s3://drivendata-competition-visiomel-public-eu/images/rqumqnfp.tif,s3://drivendata-competition-visiomel-public-as/images/rqumqnfp.tif
2,bu5xt1xm.tif,[64:66[,2,face,NO,<0.8,NO,0.264384,774102360,465947458,s3://drivendata-competition-visiomel-public-us/images/bu5xt1xm.tif,s3://drivendata-competition-visiomel-public-eu/images/bu5xt1xm.tif,s3://drivendata-competition-visiomel-public-as/images/bu5xt1xm.tif
3,dibvu7wk.tif,[62:64[,2,forearm,NaN,[2 : 4[,YES,0.226490,515827065,568174704,s3://drivendata-competition-visiomel-public-us/images/dibvu7wk.tif,s3://drivendata-competition-visiomel-public-eu/images/dibvu7wk.tif,s3://drivendata-competition-visiomel-public-as/images/dibvu7wk.tif
4,qsza4coh.tif,[90:92[,2,face,NO,[1 : 2[,NO,0.226490,1541795099,1042691978,s3://drivendata-competition-visiomel-public-us/images/qsza4coh.tif,s3://drivendata-competition-visiomel-public-eu/images/qsza4coh.tif,s3://drivendata-competition-visiomel-public-as/images/qsza4coh.tif
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,n7jd638y.tif,[68:70[,1,head/neck,NaN,[1 : 2[,NO,0.250000,1949851255,136729436,s3://drivendata-competition-visiomel-public-us/images/n7jd638y.tif,s3://drivendata-competition-visiomel-public-eu/images/n7jd638y.tif,s3://drivendata-competition-visiomel-public-as/images/n7jd638y.tif
1338,8wg601oe.tif,[66:68[,1,head/neck,NaN,<0.8,NaN,0.250000,1344207528,1603484314,s3://drivendata-competition-visiomel-public-us/images/8wg601oe.tif,s3://drivendata-competition-visiomel-public-eu/images/8wg601oe.tif,s3://drivendata-competition-visiomel-public-as/images/8wg601oe.tif
1339,1ejfd01b.tif,[80:82[,1,trunk,NaN,>=4,YES,0.250000,4292211076,1423564688,s3://drivendata-competition-visiomel-public-us/images/1ejfd01b.tif,s3://drivendata-competition-visiomel-public-eu/images/1ejfd01b.tif,s3://drivendata-competition-visiomel-public-as/images/1ejfd01b.tif
1340,2ztrjp20.tif,[42:44[,2,trunk,NaN,<0.8,NaN,0.250000,2949914817,336489430,s3://drivendata-competition-visiomel-public-us/images/2ztrjp20.tif,s3://drivendata-competition-visiomel-public-eu/images/2ztrjp20.tif,s3://drivendata-competition-visiomel-public-as/images/2ztrjp20.tif


In [23]:
def multimerge(n):
    if n == 0:
        return dfs[n]
    return dfs[n].merge(multimerge(n-1), on='filename', suffixes=(n, n-1))

In [15]:
ids = pd.DataFrame(df['filename'])
df1 = df2 = df3 = df4 = ids.copy()
df1['col1'] = 0
df2['col1'] = 1
df3['col1'] = 2
df4['col1'] = 3
dfs = [df1, df2, df3, df4]
[]

[]

In [62]:
df = dataloading.Dataset.get_dftrain('../workspace/tiles/data/processed/64/0/', 5, 42, 0)
df1 = df.groupby('tissue_id').agg({
            'tile_id': list, 'path': list})
df2 = df.drop(['path', 'tile_id'], axis=1).groupby('tissue_id').agg(lambda x: x.head(1))
dftrain = pd.merge(df1, df2, on='tissue_id').reset_index()
dftrain = pd.concat([dftrain, pd.get_dummies(dftrain['body_site']).astype(np.float32)], axis=1).drop('body_site', axis=1)
dftrain

training dataset: 1252 samples
training dataset: 58062 tiles


,tissue_id,tile_id,path,filename,relapse,split,n_tiles,is_valid,age,sex,...,nail,neck,scalp,seat,sole,thigh,toe,trunc,trunk,upper limb/shoulder
0,00shcdng,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0]","[../workspace/tiles/data/processed/64/0/00shcdng_0.jpeg, ../workspace/tiles/data/processed/64/0/00shcdng_1.jpeg, ../workspace/tiles/data/processed/64/0/00shcdng_2.jpeg, ../workspace/tiles/data/processed/64/0/00shcdng_3.jpeg, ../workspace/tiles/data/processed/64/0/00shcdng_4.jpeg, ../workspace/tiles/data/processed/64/0/00shcdng_5.jpeg, ../workspace/tiles/data/processed/64/0/00shcdng_6.jpeg, ../workspace/tiles/data/processed/64/0/00shcdng_7.jpeg, ../workspace/tiles/data/processed/64/0/00shcdng_8.jpeg, ../workspace/tiles/data/processed/64/0/00shcdng_9.jpeg, ../workspace/tiles/data/processed/6...",00shcdng.tif,0.0,3.0,48.0,0.0,0.756098,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00v6fzxt,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0]","[../workspace/tiles/data/processed/64/0/00v6fzxt_0.jpeg, ../workspace/tiles/data/processed/64/0/00v6fzxt_1.jpeg, ../workspace/tiles/data/processed/64/0/00v6fzxt_2.jpeg, ../workspace/tiles/data/processed/64/0/00v6fzxt_3.jpeg, ../workspace/tiles/data/processed/64/0/00v6fzxt_4.jpeg, ../workspace/tiles/data/processed/64/0/00v6fzxt_5.jpeg, ../workspace/tiles/data/processed/64/0/00v6fzxt_6.jpeg, ../workspace/tiles/data/processed/64/0/00v6fzxt_7.jpeg, ../workspace/tiles/data/processed/64/0/00v6fzxt_8.jpeg, ../workspace/tiles/data/processed/64/0/00v6fzxt_9.jpeg, ../workspace/tiles/data/processed/6...",00v6fzxt.tif,0.0,1.0,48.0,0.0,0.292683,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02ugc09f,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0]","[../workspace/tiles/data/processed/64/0/02ugc09f_0.jpeg, ../workspace/tiles/data/processed/64/0/02ugc09f_1.jpeg, ../workspace/tiles/data/processed/64/0/02ugc09f_2.jpeg, ../workspace/tiles/data/processed/64/0/02ugc09f_3.jpeg, ../workspace/tiles/data/processed/64/0/02ugc09f_4.jpeg, ../workspace/tiles/data/processed/64/0/02ugc09f_5.jpeg, ../workspace/tiles/data/processed/64/0/02ugc09f_6.jpeg, ../workspace/tiles/data/processed/64/0/02ugc09f_7.jpeg, ../workspace/tiles/data/processed/64/0/02ugc09f_8.jpeg, ../workspace/tiles/data/processed/64/0/02ugc09f_9.jpeg, ../workspace/tiles/data/processed/6...",02ugc09f.tif,0.0,3.0,48.0,0.0,0.463415,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,031viy0y,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0]","[../workspace/tiles/data/processed/64/0/031viy0y_0.jpeg, ../workspace/tiles/data/processed/64/0/031viy0y_1.jpeg, ../workspace/tiles/data/processed/64/0/031viy0y_2.jpeg, ../workspace/tiles/data/processed/64/0/031viy0y_3.jpeg, ../workspace/tiles/data/processed/64/0/031viy0y_4.jpeg, ../workspace/tiles/data/processed/64/0/031viy0y_5.jpeg, ../workspace/tiles/data/processed/64/0/031viy0y_6.jpeg, ../workspace/tiles/data/processed/64/0/031viy0y_7.jpeg, ../workspace/tiles/data/processed/64/0/031viy0y_8.jpeg, ../workspace/tiles/data/processed/64/0/031viy0y_9.jpeg, ../workspace/tiles/data/processed/6...",031viy0y.tif,0.0,1.

In [68]:
ds = dataloading.Dataset('../workspace/tiles/data/processed/64/0/', 5, 42, 0, 48, 48, 512, isval=False)
dl = DataLoader(ds, 1, True)
ds[0][1].shape

training dataset: 1252 samples
training dataset: 58062 tiles


torch.Size([1, 23])

In [24]:
multimerge(3)

,filename,col13,col12,col11,col10
0,w6uejlue.tif,3,3,3,3
1,tdb0zem7.tif,3,3,3,3
2,rr2jx9as.tif,3,3,3,3
3,dhaohj9d.tif,3,3,3,3
4,k85cemwe.tif,3,3,3,3
...,...,...,...,...,...
1337,8qmf9gy2.tif,3,3,3,3
1338,z5qdks0z.tif,3,3,3,3
1339,ai561bat.tif,3,3,3,3
1340,47ne6ap5.tif,3,3,3,3
